### **CNN으로 MNIST 분류하기**

합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d) 을 하나의 합성곱 층으로 보고 명명하도록 함
(맥스풀링을 별도로 명명하는 표기 방법도 존재함)

**1번 레이어 : 합성곱층(Convolutional layer)**
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

**2번 레이어 : 합성곱층(Convolutional layer)**
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

**3번 레이어 : 전결합층(Fully-Connected layer)**
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax


---
### **모델 구현하기**

1) 필요한 도구 임포트, 입력의 정의

In [1]:
import torch
import torch.nn as nn

In [2]:
# 임의의 텐서 만들기

inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))


텐서의 크기 : torch.Size([1, 1, 28, 28])


2. 합성곱층과 풀링 선언

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1) # 입력, 출력, 커널, 패딩
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool = nn.MaxPool2d(2)
# MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) 위와 동일함
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


3. 구현체를 연결하여 모델 만들기

In [6]:
out = conv1(inputs) # 32는 conv1의 out_channal이 32라서
print(out.shape) # 28 너비 28 높이는 패딩 1폭에 3x3 커널이라 크기가 보존되어서

torch.Size([1, 32, 28, 28])


In [7]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out = conv2(out)
print(out.shape) # 14 너비 14 높이는 이것도 패딩 1폭에 3x3 커널이라 크기가 보존되어서

torch.Size([1, 64, 14, 14])


In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [10]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [11]:
out = out.view(out.size(0), -1) # 배치 차원은 그대로 두고 나머지만 펼침침
print(out.shape)

torch.Size([1, 3136])


In [12]:
fc = nn.Linear(3136, 10) # input_dim, output_dim
out = fc(out)
print(out.shape)

torch.Size([1, 10])


---
### **CNN으로 MNIST 분류하기**

In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)

if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [15]:
# 하이퍼파라미터 설정

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [16]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

In [17]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [18]:
# 모델 구현

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(X)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [19]:
# CNN 모델 정의
model = CNN().to(device)

In [20]:
# 비용함수와 옵티마이저 정의의
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# 총 데이터가 60000개인데 미니배치수가 600, 배치사이즈100이므로 600 x 100 = 60000
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [22]:
for epoch in range(training_epochs):
    avg_cost=0 # 에포크 당 평균 비용 저장

    for X, Y in data_loader: # 미니 배치 단위로 데이터를 꺼냄
        X = X.to(device) # X는 이미 28x28 크기라 별도의 reshape 필요 X
        Y = Y.to(device) # Y는 원-핫 인코딩이 아닌 정수형 클래스 테이블

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225655377
[Epoch:    2] cost = 0.0630311221
[Epoch:    3] cost = 0.0462456085
[Epoch:    4] cost = 0.0373798795
[Epoch:    5] cost = 0.0312730484
[Epoch:    6] cost = 0.0258990899
[Epoch:    7] cost = 0.0217841305
[Epoch:    8] cost = 0.0182101168
[Epoch:    9] cost = 0.0159356762
[Epoch:   10] cost = 0.013302465
[Epoch:   11] cost = 0.010068506
[Epoch:   12] cost = 0.00945119094
[Epoch:   13] cost = 0.00865749642
[Epoch:   14] cost = 0.00605538301
[Epoch:   15] cost = 0.00706629548


In [52]:
# 테스트
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size)

# 정확도 계산
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for X_test, Y_test in test_loader:
        # 테스트 데이터를 모델에 입력하기 위한 준비
        X_test = X_test.view(len(X_test), 1, 28, 28).float().to(device) # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
        Y_test = Y_test.to(device) # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

        # 모델 예측 수행
        prediction = model(X_test)

        # 예측값과 실제 레이블 비교
        correct_prediction = torch.argmax(prediction, 1) == Y_test

        # 정확도 계산
        correct += correct_prediction.float().mean()
        total += 1 # 정확도 계산을 위해서 일치하는 예측의 평균값을 구함
        
    # 정확도 출력
    accuracy = correct / total
    print('Accuracy:', accuracy.item())

Accuracy: 0.09940000623464584
